In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from   sklearn.compose         import *
from   sklearn.experimental    import enable_iterative_imputer
from sklearn.preprocessing import OrdinalEncoder
from   sklearn.impute          import *
from   sklearn.linear_model    import Lasso, Ridge
from sklearn.ensemble        import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from   sklearn.metrics         import mean_squared_error,mean_absolute_error
from   sklearn.model_selection import train_test_split
from   sklearn.pipeline        import Pipeline
from   sklearn.preprocessing   import *

Ricky Zhang / MSDS 699 Final Project: Zillow’s Home Value Prediction: Zestimate
----------

Research Question / Hypothesis
----




__The research question of this project is to predict the log-error between Zillow's estimate and the actual sale price, given all the features of a home.__

__Obviously, the housing price is highly dependent on some of the most prominent features such as SQFT, # of bedrooms, # of bathrooms, etc. However, we still have numerous other interesting features of homes in this dataset, like AC type, garage SQFT, fire place count. I'm very excited to find out how they are correlated with the price__

Load Data
-----

In [2]:
### data source: https://www.kaggle.com/c/zillow-prize-1/data?select=train_2017.csv 
### https://www.kaggle.com/c/zillow-prize-1/data?select=properties_2017.csv
df = pd.read_csv('zillow.csv')
### the dataframe I'm working on is downsampled to 3,000 rows for computation efficiency

In [6]:
X = df.drop('logerror',axis=1)
y = df['logerror'].copy() ###log_error is the target variable

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
## use cv on train set to choose model, final test on test set

Preprocessing
----

In [8]:
drop_cols = [] ###for columns has more than half missing values, I simply get rid of them
for col in df.columns.tolist():
    if df[col].isnull().sum() / df['parcelid'].count() > 0.5:
        drop_cols.append(col)

In [9]:
df.drop(drop_cols,axis=1).info() ###take another look after dropping these variables

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 32 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   parcelid                      3000 non-null   int64  
 1   airconditioningtypeid         3000 non-null   float64
 2   bathroomcnt                   3000 non-null   float64
 3   bedroomcnt                    3000 non-null   float64
 4   buildingqualitytypeid         2635 non-null   float64
 5   calculatedbathnbr             2998 non-null   float64
 6   calculatedfinishedsquarefeet  3000 non-null   float64
 7   finishedsquarefeet12          2961 non-null   float64
 8   fips                          3000 non-null   float64
 9   fullbathcnt                   2998 non-null   float64
 10  heatingorsystemtypeid         2819 non-null   float64
 11  latitude                      3000 non-null   float64
 12  longitude                     3000 non-null   float64
 13  lot

In [10]:
drop_cols.append('transactiondate') ##get rid of transaction date too

In [19]:
### Though the fact that only 3 columns are type object from df.info(), there're actually more categoricals already
### in numeric form. We would also want to one-hot encode these to pass into linear models
cat = ['airconditioningtypeid','buildingqualitytypeid','heatingorsystemtypeid','propertyzoningdesc',
       'propertycountylandusecode','propertylandusetypeid']

In [20]:
all_cols = X.columns.tolist()

In [21]:
### remaining are continuous varibles
remaining = [x for x in all_cols if x not in cat and x not in drop_cols]

In [22]:
cat_pipe = Pipeline([('ohe', OneHotEncoder(handle_unknown='ignore')),
                     ('imputer', SimpleImputer(strategy='most_frequent', add_indicator=True))])

con_pipe = Pipeline([('imputer', SimpleImputer(missing_values=np.nan,strategy='median', add_indicator=True)),
                     ('scaler', StandardScaler())
                      ])

preprocessing = ColumnTransformer([('categorical', cat_pipe, cat),
                                   ('continuous', con_pipe, remaining),
                                   ('drop', 'drop', drop_cols)
                                   ])

Algorithms & Search
-------

Methodology: grid search for optimal hyperparameters for Lasso & Ridge to get a baseline model, then fit a RF model, compare two models performance to decide final model

In [34]:
###use two linear algorithms as baseline models
linear_algorithms = {
    
    Lasso:{'alpha':(5, 0.5, 0.05, 0.005, 0.0005, 1, 0.1, 0.01,0.001,0.0001),
           'normalize':(True,False)},
    
    Ridge:{'normalize':(True,False),
           'alpha':(5, 0.5, 0.05, 0.005, 0.0005, 1, 0.1, 0.01,0.001,0.0001)}
    
}

In [27]:
###find optimal hyperparameters for RF and compare to baseline linear model
rf = {
    
    RandomForestRegressor:{'n_estimators':np.arange(10,500,20), ##with more trees, bagged trees see more of training data
                            'criterion' : ("mse", "mae"), 
                            'max_depth':np.arange(5,100,5),
                            'min_samples_split':np.arange(1,12,2),
                            'min_samples_leaf':np.arange(1,50,1),
                            'max_features' : ("auto", "sqrt", "log2")##all these 5 hp limit overfitting for each tree,
                                                                     ##therefore increasing generality
                            }
    
}

In [35]:
###Grid search CV to tune hp for linear model
for algo,hp in linear_algorithms.items():
    pipe = Pipeline([('preprocessing', preprocessing), 
                     ('rgr',     algo())])
    
    d=dict()
    
    for k,v in hp.items():
        s = 'rgr__'+k
        d[s] = v
        
    linear_rgr_grid_cv = GridSearchCV(estimator=pipe, 
                              param_grid=d,
                              cv=5,
                              verbose=True)  
    linear_rgr_grid_cv.fit(X_train,y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:513: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.03298179255704525, tolerance: 0.0037698047637306008
  model = cd_fast.sparse_enet_coordinate_descent(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:513: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.010035615830695122, tolerance: 0.0038055665482468032
  model = cd_fast.sparse_enet_coordinate_descent(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:513: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.01405760581299731, tolerance: 0.0038230319177292196
  model = cd_fast.sparse_enet_coordinate_descent(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coor

Fitting 5 folds for each of 20 candidates, totalling 100 fits


In [48]:
##RF takes much more time to train, so we use RandomizedSearchCV here
for algo,hp in rf.items():
    pipe = Pipeline([('preprocessing', preprocessing), 
                     ('rgr',     algo())])
    
    d=dict()
    
    for k,v in hp.items():
        s = 'rgr__'+k
        d[s] = v
        
    rf_rgr_rand_cv = RandomizedSearchCV(estimator=pipe, 
                              param_distributions=d,
                              cv=5,
                              n_jobs=-1,
                              verbose=True)  
    rf_rgr_rand_cv.fit(X_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:918: UserWarning: One or more of the test scores are non-finite: [-0.00233394  0.00415299  0.00475883         nan -0.00057651  0.00080045
         nan  0.00063649         nan  0.00097041]
  warnings.warn(


In [37]:
linear_rgr_grid_cv.best_estimator_

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('categorical',
                                                  Pipeline(steps=[('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore')),
                                                                  ('imputer',
                                                                   SimpleImputer(add_indicator=True,
                                                                                 strategy='most_frequent'))]),
                                                  ['airconditioningtypeid',
                                                   'buildingqualitytypeid',
                                                   'heatingorsystemtypeid',
                                                   'propertyzoningdesc',
                                                   'propertycountylandusecode',
                                                

In [49]:
rf_rgr_rand_cv.best_estimator_

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('categorical',
                                                  Pipeline(steps=[('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore')),
                                                                  ('imputer',
                                                                   SimpleImputer(add_indicator=True,
                                                                                 strategy='most_frequent'))]),
                                                  ['airconditioningtypeid',
                                                   'buildingqualitytypeid',
                                                   'heatingorsystemtypeid',
                                                   'propertyzoningdesc',
                                                   'propertycountylandusecode',
                                                

Evaluation Metrics
--------

Here we use best estimator from linear model and RF to perform CV on training set; we use both MSE & MAE to evaluate performance, because they both measure how off our prediction is from true response, but MSE is more sensitive to outliers.

In [52]:
from sklearn.model_selection import cross_val_score
cv_mse_score = -cross_val_score(linear_rgr_grid_cv.best_estimator_, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
cv_mae_score = -cross_val_score(linear_rgr_grid_cv.best_estimator_, X_train, y_train, cv=5, scoring='neg_mean_absolute_error')
print("Mean 5-Fold mae: {}".format(np.mean(cv_mae_score)))
print("Mean 5-Fold mse: {}".format(np.mean(cv_mse_score)))

Mean 5-Fold mae: 0.06300257554302456
Mean 5-Fold mse: 0.019010426357808134


In [53]:
from sklearn.model_selection import cross_val_score
cv_mse_score = -cross_val_score(rf_rgr_rand_cv.best_estimator_, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
cv_mae_score = -cross_val_score(rf_rgr_rand_cv.best_estimator_, X_train, y_train, cv=5, scoring='neg_mean_absolute_error')
print("Mean 5-Fold mae: {}".format(np.mean(cv_mae_score)))
print("Mean 5-Fold mse: {}".format(np.mean(cv_mse_score)))

Mean 5-Fold mae: 0.061962005298446754
Mean 5-Fold mse: 0.018781846263504084


Random forest do better in terms of both metrics -> smaller error in prediction; RF as final model

Final Model
--------

In [62]:
###Use Random Forest as final model
hp = {
    'max_depth': 50,
    'max_features': 'sqrt',
    'min_samples_leaf': 14,
    'min_samples_split': 7,
    'n_estimators': 450
}
pipe = Pipeline([('preprocessing', preprocessing), 
                     ('rgr',  RandomForestRegressor(**hp))])
pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)

print("Test MAE: {}".format(mean_absolute_error(y_pred,y_test)))
print("Test MSE: {}".format(mean_squared_error(y_pred,y_test)))

Test MAE: 0.056731637508977265
Test MSE: 0.011979488269715103


We used final model to test on test set, and yields even better results than CV on training set, which confirms the model has decent generality and ready for production.<br>

In our case, performance of baseline model Ridge is very close to our final model; this is because many independent variables do have a linear relationship with response, and response variable follows normal distribution. Therefore, a linear model is suitable for this dataset. However, Random Forest still performs slightly better, as its setup is more complex and suitable for every case, but it takes much more computation power to train as well.<br>

For this problem, we predicted log-error between Zillow's estimate and actual price. In my opinion, the business problem here is, if we can develop a model to predict this error, by examining our model and feature importance, we can understand how Zillow can improve their model for price prediction.<br>

Next step: We understand that which factors are causing the error in Zillow's estimate, and we then improve Zillow's model to get a better estimate on top of our analysis